# Import Packages

In [1]:
import pandas as pd

# Load Data

Data is downloaded from MongoDB and processed to raw csv. Needed to import clicks separately since click event contains a `data.item` attribute that cannot be exported when used with other data(?).

> Needs to be fixed. Also click event should fill `items` with a list (but just containing a single entry: the item which was clicked on)


Transactions are retrieved from SAP Commerce via `flexible search query`.

In [2]:
VERSION = '20220327'

ev_df = pd.read_csv(f'../../data/raw/evidence_{VERSION}.csv')
ev_click_df = pd.read_csv(f'../../data/raw/evidence_click_{VERSION}.csv')
user_df = pd.read_csv(f'../../data/raw/user_{VERSION}.csv')
order_entries_df = pd.read_csv(f'../../data/raw/order_entries_{VERSION}.csv')
item_df = pd.read_csv(f'../../data/raw/items_{VERSION}.csv')

print(f"ev_df shape: {ev_df.shape}")
print(f"ev_click_df shape: {ev_click_df.shape}")
print(f"user_df shape: {user_df.shape}")
print(f"order_entries_df shape: {order_entries_df.shape}")
print(f"item_df shape: {item_df.shape}")

FileNotFoundError: [Errno 2] No such file or directory: '../../data/raw/items_20220327.csv'

# Evidence

In [ ]:
def rename_columns(df):
    return df.rename(columns={'data.duration': 'duration', 'data.items': 'items', 'data.item': 'items', 'device_info.is_mobile': 'is_mobile'})

ev_df = rename_columns(ev_df)
ev_click_df = rename_columns(ev_click_df)

# Convert click evidence item to a list to match with reco_view (also contains a list of items)
ev_click_df['items'] = ev_click_df['items'].apply(lambda a: [str(a)])

# Since "main" evidence also contains the "click" events (duplicate but without the item attribute) we need to remove those
ev_df = ev_df.drop(ev_df[ev_df['name']=='click'].index)

# Add click evidence to "main" evidence (needed to be done manually since cannot be exported together with normal evidence)
ev_df = pd.concat([ev_df, ev_click_df])

In [ ]:
# Bug: there are missing recommendations for some products... IF DROPPED RANDOMIZATION ACROSS CONTROL/TREAT NOT FULFILLED!!!
# ev_df = ev_df.drop(ev_df[ev_df['items']=='[]'].index)

In [ ]:
# First, we only want to keep product details page visits, search & orders.
ev_df = ev_df[(ev_df['path'].str.contains("/c/")) | (ev_df['path'].str.contains("/p/")) | (ev_df['path'].str.contains("search")) | (ev_df['path'].str.contains("order"))]
ev_df.loc[(ev_df['path'].str.contains('/c/')) & (~ev_df['path'].str.contains('/p/')), 'name'] = 'category_search'
ev_df.loc[(ev_df['path'].str.contains("/p/")) & (ev_df['name']=='visit'), 'name'] = 'product_view'
ev_df.loc[(ev_df['path'].str.contains("/p/")) & (ev_df['name']=='view'), 'name'] = 'reco_view'
ev_df.loc[(ev_df['path'].str.contains("/p/")) & (ev_df['name']=='unview'), 'name'] = 'reco_unview'
ev_df.loc[(ev_df['path'].str.contains("/search")) & (ev_df['name']=='visit'), 'name'] = 'search'

In [ ]:
# -> we drop the unclear is_mobile assignments
keep_devices = ['[false,false,false]', '[true,true,true]']
ev_df = ev_df[ev_df['is_mobile'].isin(keep_devices)]
# -> transform is_mobile in 0/1
target_device_values = [0, 1]
mob_dict = dict(zip(keep_devices, target_device_values))
ev_df['is_mobile'] = ev_df['is_mobile'].map(mob_dict)

In [ ]:
# Merge Users
ev_df = ev_df.merge(user_df, how='left', left_on='user_uid', right_on='_id')
ev_df = ev_df.rename(columns={'groups.split1': 'group'})
ev_df = ev_df[ev_df['group'].notna()] # users only get assigned to group if they visit PDP

map_group = {'cf_ib': 'treatment', 'random': 'control'}
ev_df['group'] = ev_df['group'].map(map_group)

In [ ]:
ev_df = ev_df.sort_values(by='timestamp', ascending=False)

# Orders
This data comes from SAP Commerce

In [ ]:
print(f"order_entries_df shape: {order_entries_df.shape}")

In [ ]:
ev_df['order_code'] = ev_df[ev_df['path'].str.contains("order")]['path'].apply(lambda o: o[-8:]).astype('int64')

In [ ]:
ev_df = ev_df.merge(order_entries_df, how='left', left_on='order_code', right_on='ORDER_CODE')

In [ ]:
ev_df.loc[ev_df['order_code'].notna(), 'name'] = 'order_entry'

In [ ]:
ev_df = ev_df.drop(columns=['_id', 'ORDER_CODE', 'ORDER_ENTRY_TOTAL_PRICE', 'ORDER_DELIVERY_COST', 'USER_UID', 'ORDER_DATE'])
ev_df['duration'] = (ev_df['duration'].fillna(0)/1000).astype('float32')
ev_df['order_code'] = ev_df['order_code'].fillna(0).astype('int64')
ev_df['PRODUCT_CODE'] = ev_df['PRODUCT_CODE'].fillna(0).astype('int64')
ev_df.columns= ev_df.columns.str.strip().str.lower()

## Order Dataframe
Contains aggregated info on orders.

In [ ]:
order_df = ev_df[ev_df['name']=='order_entry']
order_df = order_df.groupby(['user_uid', 'order_code']).max().sort_values(by='timestamp', ascending=False)
order_df = order_df.drop(columns=['duration', 'items', 'path', 'product_code', 'quantity', 'product_price'])
order_df['name'] = 'order'

## Order Entries DataFrame

In [ ]:
order_entry_df = ev_df[ev_df['name']=='order_entry']
order_entry_df = order_entry_df.drop(columns=['duration', 'items', 'path'])

In [ ]:
# merge count of order_entries per customer
order_df = order_df.merge(order_entry_df.groupby('user_uid')['quantity'].count(), how='left', on='user_uid')
order_df = order_df.rename(columns={'quantity': 'entry_qty'})

# Search Behavior

In [ ]:
f_search_df = ev_df[ev_df['name']=='search']
f_search_df.groupby(['is_mobile', 'group'])['user_uid'].count()

In [ ]:
c_search_df = ev_df[ev_df['name']=='category_search']
c_search_df.groupby(['is_mobile', 'group'])['user_uid'].count()

# User DataFrame

Aggregate info to a user dataframe.

In [ ]:
user_df = pd.DataFrame(ev_df.groupby(['user_uid', 'group', 'is_mobile'])['user_uid', 'group', 'is_mobile'].first()).reset_index(drop=True)

# Remove Duplicates
user_df = user_df.drop(user_df[user_df.duplicated(subset=['user_uid'], keep=False)].index)

# Merge number of category search calls
user_df = user_df.merge(c_search_df.groupby(['user_uid'])['name'].count(), how='left', on='user_uid').rename(columns={'name': 'n_category_search'})

# Relevant User Timestamps
user_df = user_df.merge(ev_df[ev_df['name']=='reco_view'].groupby(['user_uid'])['timestamp'].agg(t_first_reco_view='min'), how='left', on='user_uid')
user_df = user_df.merge(ev_df.groupby('user_uid')['timestamp'].agg(t_0='min', t_T='max'), how='left', on='user_uid')

## Consideration Set

In [ ]:
def convert_path_to_item_id(s):
    try:
        s = s[s.find('/p/')+3:s.find('/p/')+21]
        s = int(s)
        if s>9999:
            return 0
        else:
            return int(s[-7:])
    except:
        return s

ev_df.loc[ev_df['name']=='product_view', 'items'] = ev_df[ev_df['name']=='product_view']['path'].apply(lambda a: convert_path_to_item_id(a))

In [ ]:
# Drop variants (only product_view) we keep reco_view/reco_unview/click
evidence_df = ev_df.drop(ev_df[(ev_df['name']=='product_view') & (ev_df['items']==0)].index)
user_df = user_df.merge(evidence_df[evidence_df['name']=='product_view'].groupby(['user_uid'])['items'].agg(product_views='count', css='nunique'), how='left', on='user_uid')

# Items

In [ ]:
item_cat_df = item_df.groupby('PRODUCT_CODE')['CATEGORY_SOURCE'].unique()
# Remove "Warengruppe"
# item_cat_df = item_cat_df.apply(lambda x: [i for i in x if not i.isdigit()])
item_cat_df = item_cat_df.apply(lambda x: [i for i in x if not '2' in i])

item_cat_df


In [ ]:
d = {}
for i in item_cat_df.index.values:
    d[i] = []
    for j in item_cat_df.index.values:
        d[i].append(int(len(set.intersection(set(item_cat_df[i]), set(item_cat_df[j])))>0))

In [ ]:
item_sim = pd.DataFrame(d, index=item_cat_df.index.values)
item_sim

# Output

In [ ]:
group = 'treatment'
mobile= 1
len(ev_df[(ev_df.is_mobile==mobile) & (ev_df.group==group) & (ev_df.name=='click')])/len(ev_df[(ev_df.is_mobile==mobile) & (ev_df.group==group) & (ev_df.name=='reco_view')])

In [ ]:
ev_df[ev_df.user_uid=='62402ff15c796c58d2f5cc4c']

In [ ]:
ev_df.to_csv(f'../../data/processed/evidence_{VERSION}.csv', index=False)
order_df.to_csv(f'../../data/processed/order_{VERSION}.csv', index=False)
order_entry_df.to_csv(f'../../data/processed/order_entry_{VERSION}.csv', index=False)
user_df.to_csv(f'../../data/processed/user_{VERSION}.csv', index=False)